In [45]:
using Base.Threads;
using Plots, LinearAlgebra, Distributions, Compose, LaTeXStrings, JLD2, Pkg, SparseArrays;
using ITensors, ITensorMPS,InvertedIndices,Einsum, BenchmarkTools, KrylovKit, Arpack;

## Tensors

$\quad$ The first step is to learn how to write tensors, with the bond dimension(s) and real dimension(s). 

$\quad$ We will denote bond dimensions by $D$, and physical dimensions by $d$.

$\quad$ We build a tensor by making an object that has $N$ indices, each index will carry a $D \times D$ dimensional object.

In [2]:
""" This is what a tensor will look like"""

N = 5; #This is the total number of indices. This corresponds to the rank of the tensor
D = 2; #This is the number of values that each index can take.
tensor_size = ntuple(_ -> D, N); #This creates a set like (D,D,D,D) for N = 4
tensor = fill(0.0, tensor_size...); #This creates a tensor of dimensions (D,D,D,D) for N = 4 and fills each of the element with 0.

### Matrix Product States

$\quad$ Now we construct a $3-$ tensor, which is the building block of an MPS (of bond dimension $d$ and physical dimension $D$). 

$\quad$ A general MPS can be constructed by a sequence of $3$ tensors, where 

$\quad$ 1. The left-most tensor has dimensions $(1,d, D)$
 
$\quad$ 2. The right-most tensor has dimension $(D,d,1)$ and 

$\quad$ 3. All those in between have dimensions $(D,d,D)$.

In [3]:
""" Here we choose the bond dimension to be same across all bonds, for simplicity. It can be changed easily """

M = 10;         # Number of sites.
d = 3;          # Physical dimension at each site.
D = 20;         # Bond dimension.

generic_mps = Any[];

push!(generic_mps,rand(Normal(0,1),(1,d,D)));

for i in 2:M-1

    push!(generic_mps,rand(Normal(0,1),(D,d,D)));

end

push!(generic_mps,rand(Normal(0,1),(D,d,1)));

$\quad$ The next step is to turn a $3-$ tensor into a $2-$ tensor (matrix) by $\textit{reshaping}$. We do this to eventually be able to perform SVD.

$\quad$ The goal of performing SVD is to turn the MPS into a left (or right) canonical form, thereby also normalizing it.

In [4]:
"""
    This general function reshapes a tensor according to the input provided by the user. 

    The inputs are the tensor, the rank of the tensor and the indices that need to be fused into one index.

    In the even that fusions have to be done, then this function has to be used successively 
    
"""

function tensor_reshape_fuse(tensor_input,tensor_rank,tensor_indices_to_fuse;fused_location = 1)
    temp_tensor = copy(tensor_input); # Create a copy of the tensor that is provided as input

    initial_rank = tensor_rank; #This is the rank of the input tensor
    final_rank = initial_rank - length(tensor_indices_to_fuse) + 1; # This is the final rank to be obtained after reshaping


    dims2 = prod(size(temp_tensor)[i] for i in tensor_indices_to_fuse); # This is the dimension of the index obtained by
                                                                        # fusing the indices that are provided as argument
                                                                        # via "tensor_indices_to fuse"

    res_tup = ntuple(_-> 0,final_rank);  # This is the tuple where the final shape will be stored.


    res_tup = setindex(res_tup, dims2,fused_location); # "fused_location" defines the index (in the final tensor) to which the fused
                                                       # index is ultimately mapped 
    
    initial_indices = vec(1:1:tensor_rank); # This is the list of initial indices of the tensor
    initial_indices_remain = Any[]; # This is the vector that stores the final set of indices (that are not fused)
                                    # Note that the index numbering is the same as the original tensor.
                                    # For example, if a tensor of has indices (i1,i2,i3,i4) and we want to fuse (i2,i3)
                                    # then intial_indices_remain will store (i1,i2)

    

    for i in 1:tensor_rank
        if i in tensor_indices_to_fuse
            nothing;
        else 
            push!(initial_indices_remain,initial_indices[i])
        end
    end
    push!(initial_indices_remain,-1);
    initial_indices_remain = sort(initial_indices_remain);

    initial_indices_remain[1], initial_indices_remain[fused_location] = initial_indices_remain[fused_location], initial_indices_remain[1]

    # The following loop sets the tuple that stores the dimensions of the final tensor
    # The dimensions of the fused indices are stored at fused_location
    # For all the other locations, we just store the dimensions of the remaining indices of the
    # original tensor.

    tsize = size(temp_tensor);
    for i in 1:final_rank
        if initial_indices_remain[i]!=-1
            res_tup = setindex(res_tup,tsize[initial_indices_remain[i]],i)
        end
    end

    # The final (easiest) step is to reshape the tensor.

    temp_tensor = reshape(temp_tensor,res_tup)

    return temp_tensor

end

tensor_reshape_fuse (generic function with 1 method)

In [5]:
"""
    This function is built to take a particular tensor index, and split it into multiple tensor indices of specified dimensions. 
    Again, if multiple indices have to be split then this has to be applied multiple times.
"""

function tensor_reshape_split(tensor_input,index_to_split,pos_indices,dimensions_of_output_indices)
    temp_tensor = copy(tensor_input); # Create a copy of the tensor that is provided as input

    dims_in = size(temp_tensor)[index_to_split]; # This is the dimensions of the input tensor index that has to be split

    dims_out = dimensions_of_output_indices; # This is the dimensions of the output indices

    initial_indices = collect(size(temp_tensor))[Not(index_to_split)]; #The dimensions of the initial tensor, except the index which has to be splot

    initial_rank = length(size(temp_tensor)); #The rank of the initial tensor

    final_rank = length(size(temp_tensor)) - 1 + length(pos_indices); #The rank of the final tensor

    res_tup = ntuple(_->0, final_rank) # This tuple stores the dimensions of the final tensor

    # The tuple is then modified by storing the dimensions of the split index in the locations specified in "pos_indices", and the remaining
    # ones in the remaining locations.

    j1 = 1;
    j2 = 1;
    for i in 1:final_rank
        if i in pos_indices
            res_tup = setindex(res_tup,dims_out[j1],i)
            j1 += 1;
        else
            res_tup = setindex(res_tup,initial_indices[j2],i)
            j2 += 1;
        end
    end

    # The final (easiest) step is to reshape the tensor.

    temp_tensor = reshape(temp_tensor,res_tup)

    return temp_tensor

end

tensor_reshape_split (generic function with 1 method)

$\quad$ The next step is to evaluate the left and right canonical forms of the MPS, by using SVD

$\quad$ For this, we shall use $\textit{reshaping}$ procedure, but without the more general functions because they are not required for these specific cases.

$\quad$ The steps are as follows:

$\quad$ 1. First, reshape every $3-$ tensor in the MPS into a $2-$ tensor (i.e. matrix) by fusing the left and central leg.

$\quad$ 2. Then perform an SVD to break the matrix into $U$, $S$ and $V^\dagger$. Here $U$ actually corresponds to the fused legs.

$\quad$ 3. Then split $U$ into a $3-$ tensor. This $U$ now connects to $S V^\dagger$, which then connect to the next tensor.

$\quad$ 4. Identify $U$ as the left canonical element of the MPS at that site. Then multiply $S V^\dagger$ to the tensor at the next site (by using Einstein summation convention for tensor multiplication between a $2-$ tensor and a $3-$ tensor)

In [6]:
""" 
    This function generates the left canonical form, and then makes sure that it is normalized.

"""

function left_canonical(MPS_state)
    mps_st = copy(MPS_state);

    N = length(mps_st); # This is the number of lattice site indices that the MPS corresponds to.

    for i in 1:N

        T = mps_st[i]; # Picking the i th tensor

        T_res = tensor_reshape_fuse(T,3,[1,2]); # Reshape the tensor into a matrix by fusing its' first and second indices

        U, S, V = svd(T_res,full=false); # Perform the SVD to extract U, S and V

        Vt = V'; # Store V transpose, since that is what we shall be using

        mps_st[i] = tensor_reshape_split(U,1,[1,2],[size(T)[1],size(T)[2]]);

        SV = Diagonal(S)*Vt;

        if i < N - 1
            mps_next = mps_st[i+1]
            @einsum nsite_update[i,j,k] :=  SV[i,l]*mps_next[l,j,k];
            mps_st[i + 1] = nsite_update;
        end

    end

    return mps_st 

end

left_canonical (generic function with 1 method)

In [7]:
""" 
    This function generates the right canonical form, and then makes sure that it is normalized.

"""

function right_canonical(MPS_state)
    mps_st = copy(MPS_state);

    N = length(mps_st); # This is the number of lattice site indices that the MPS corresponds to.

    for i in N:-1:1
        

        T = mps_st[i]; # Picking the i th tensor

        T_res = tensor_reshape_fuse(T,3,[2,3];fused_location=2); # Reshape the tensor into a matrix by fusing its' first and second indices

        U, S, V = svd(T_res,full=false); # Perform the SVD to extract U, S and V 

        mps_st[i] = tensor_reshape_split(V',2,[2,3],[size(T)[2],size(T)[3]]); # Reshape V transpose to store it as the 3-tensor at that location

        US = U*Diagonal(S); # Evaluate U*S as that has to combine with the previous tensor

        if i > 1
            mps_prev = mps_st[i-1]
            @einsum nsite_update[i,j,k] :=  mps_prev[i,j,l]*US[l,k];
            mps_st[i - 1] = nsite_update;
        end

    end

    return mps_st 

end

right_canonical (generic function with 1 method)

$\quad$ In this step, we verify whether the (left and right cannonical) MPS that we have generated is accurate or not.

$\quad$ A simple check is to see if all the tensors in the left (right) cannonical form become left (right) normalized.

$\quad$ The normalisation constraint is imposed by taking each tensor $M_{\alpha \beta \gamma}$ and demanding that 

$\quad$ $\sum_{\alpha, \beta} M_{\alpha \beta \gamma}^{\dagger} M_{\alpha \beta \gamma'} = \delta_{\gamma \gamma'}$. Note that $M^{\dagger}_{a b c} = M^{*}_{c b a}$. This is $\textit{left normalization}$.

$\quad$ For $\textit{right normalization}$ , we impose $\sum M^{\dagger}_{\alpha \beta \gamma} M_{\alpha' \beta \gamma} = \delta_{\alpha \alpha'}$.

In [8]:
""" 
    Here we check the normalization condition on each tensor for the right canonical form of the MPS

"""

N = 10;

mps_right_c = right_canonical(generic_mps)

deficit = 0.0;
for l in 1:N
    mps_dag = permutedims(conj.(mps_right_c[l]),(3,2,1)) #right-top-left

    B = mps_right_c[l];

    @einsum MPSprod[i,m] := B[i,j,k]*mps_dag[k,j,m] #top-bottom

    A = I + fill(0.0,size(MPSprod))
    
    deficit += maximum(abs.(MPSprod-A))
end
print("Deficit = " ,deficit)

Deficit = 1.609823385706477e-14

In [9]:
""" 
    Here we check the normalization condition on each tensor for the left canonical form of the MPS

"""

N = 10;

mps_left_c = left_canonical(generic_mps)

deficit = 0.0;
for l in 1:N
    mps_dag = permutedims(conj.(mps_left_c[l]),(3,2,1)) #right-top-left

    B = mps_left_c[l];

    @einsum MPSprod[i,m] := mps_dag[i,j,k]*B[k,j,m] #top-bottom

    A = I + fill(0.0,size(MPSprod))
    
    deficit += maximum(abs.(MPSprod-A))
end
print("Deficit = " ,deficit)

Deficit = 1.7430501486614958e-14

In [10]:
"""
    This function performs a left canonical transformation on the MPS and then follows it up with right canonical transformation. This should give the
    most compact representation of the MPS with the least bond dimensions.
    
"""

function mixed_canonical(MPS_state)

    mps_temp = MPS_state;

    mps_left = left_canonical(mps_temp);

    mps_mixed = right_canonical(mps_left);

    return mps_mixed
end

mixed_canonical (generic function with 1 method)

#### MPS operation : Closing the Zipper

$\quad$ Here we describe the ``closing the zipper'' method that is used to efficiently compute norms and inner products. 

$\quad$ The first part will deal with inner products of states.

In [11]:
"""
    The first step is to write a function to generate a random MPS for ease of calculation

"""

function rand_MPS(N,d,D; d1 = Normal(0,1))

    generic_mps = Any[];

    push!(generic_mps,rand(d1,(1,d,D)));

    for i in 2:N-1

        push!(generic_mps,rand(d1,(D,d,D)));

    end

    push!(generic_mps,rand(d1,(D,d,1)));

    return generic_mps
end

rand_MPS (generic function with 1 method)

We first discuss the process of closing the zipper for states:

$\quad$ Let us consider that we have two states $\ket{\psi}$ and $\ket{\phi}$, which may have different bond dimensions. We evaluate $\braket{\phi\vert\psi}$

$\quad$ The basic operation ``closing the zipper'' is to contract $3$ indices, between the $n^{\text{th}}$ tensor of the MPS of $\ket{\psi}$ (denoted by $A_{n}$) and that of $\ket{\phi}$ (denoted by $B^{\dagger}_{n}$). 

$\quad$ Two of the indices are connected to the $2-$ tensor that has stored the result of the process till $n - 1$ (this is $C_{n - 1}$), and the remaining contraction is between the physical index legs of $\ket{\psi}$ and $\ket{\psi}$, which have dimension $d$. 

In [12]:
function zip_left(Cn_minus_1, An, Bn)
    
    # We count the legs from left to right. Thus, the first index of Cn_minus_1 must have bond dimension same as An, and the second index should
    # have bond dimension same as Bn

    Cn1 = Cn_minus_1;

    @einsum M1[i,j,l] := Bn[i,j,k]*Cn1[k,l]

    @einsum M2[i,l] := M1[i,j,p]*An[p,j,l]

    return M2
    
end

zip_left (generic function with 1 method)

$\quad$ To test out this code, we take two random MPS states and try to determine what their inner product is. We use mixed canonical form to do this.

$\quad$ Turns out that this gives us a number, as expected (yay! it works!). Taking both states to be the same gives $1$, since mixed canonical forms are normalized.

In [13]:
""" 
    
    Testing Inner product via (left) closing the zipper method 

"""

N = 10;
d = 2;
D1 = 20;
D2 = 10;

M_norm = fill(1.0,(1,1));

mps_1 = mixed_canonical(rand_MPS(N,d,D1));

mps_2 = mixed_canonical(rand_MPS(N,d,D2));

for p in 1:N

    A_n = mps_1[p];

    B_n = mps_2[p];

    B_n_dag = permutedims(conj.(B_n),(3,2,1));

    M_norm = zip_left(M_norm, A_n, B_n_dag);

end

print(M_norm[1])

0.013436658350211662

In [14]:
"""
    Here we write a function that takes two MPS states and gives their inner product

"""

function inner_product(mps_state_1, mps_state_2)

    mps_1 = mps_state_2 # This is the ket state.

    mps_2 = mps_state_1 # This is the bra state

    M_norm = fill(1.0,(1,1)); # This is C_1, which is just a 1 x 1 matrix with entry 1.0

    N = length(mps_state_1); # This is the number of indices, and these must be the same.

    for p in 1:N

        A_n = mps_1[p];
    
        B_n = mps_2[p];
    
        B_n_dag = permutedims(conj.(B_n),(3,2,1));
    
        M_norm = zip_left(M_norm, A_n, B_n_dag);
    
    end

    return M_norm[1]

end

inner_product (generic function with 1 method)

#### State to MPS

$\quad$ The final part of this discussion on MPS (before going to MPO and DMRG) is to write a full state in the Hilbert space in MPS notation

$\quad$ This will be the exact transformation, thus giving us unequal bond dimensions.

$\quad$ The first step is to write the state as an $d^N$ dimensional vector. We denote this by $\Psi_{N}$.

$\quad$ The next step is to reshape $\Psi_{N}$ into a $d \times d^{N-1}$ matrix. 

$\quad$ On this tensor, we perform SVD which gives a $d \times d$ matrix $U$, which is then treated as the first local tensor of the MPS (call it $A_1$).

$\quad$ The remainder is the $d \times d^{N-1}$ dimensional matrix.  Reshape this into a $d^2 \times d^{N-2} \equiv \psi_1$ matrix, and then SVD it.

$\quad$ This gives a $d^2 \times d^2$ matrix $U$, which is reshaped into $d \times d \times d^2$ tensor. This is the second tensor of the MPS, which we call $A_2$. 

$\quad$ Note that the size of the left leg of $A_{n}$ is equal to the size of right leg of $A_{n - 1}$. The mid leg always has size $d$. This fixes the right leg as well.

$\quad$ $\textcolor{blue}{The\;\;Key\;\;Point:}$ When reshaping $\psi_{n} = S V^\dagger$ (at the $n^\text{th}$ step), the dimensions are determined as $(dim(S)*d, dim(\psi_{n-1})/d)$.

In [15]:
function state_to_mps(state_vector, N)

    st_vec = state_vector # This is the d^N dimensional vector 

    d = Int(length(st_vec)^(1/N)); # The dimension at a single site

    psi = reshape(st_vec,(d,d^(N-1))); # First step is to isolate the first index.

    mps_state = Any[];

    left_bond_dim = 1; # The bond dimension of the left most tensor is 1 (i.e. a dummy index) on its' left most leg

    for k in 1:N-1
        #print(size(psi),"     ");

        U, S, V = svd(psi); # Perform an SVD of the state matrix

        mps_temp = reshape(U,(left_bond_dim,d,size(S,1))); # The matrix U (which is on the left of SVD) is reshaped into the tensor at site k

        push!(mps_state, mps_temp); # Store it into the MPS

        left_bond_dim = size(S,1); # This is the left bond dimension for the next tensor

        psi_1 = Diagonal(S)*V'; # This is the matrix for the remaining part of the original vector

        #print(size(psi_1),"     ");
        if k == N-1 # This is the step that constructs the tensor at the right end of the MPS
            mps_temp = reshape(psi_1, left_bond_dim[1],d,1);
            push!(mps_state,mps_temp)
        else
             # This is the step where the the remaining part vector is appropriately reshaped.
             # For this, note that we reshape the remaining part as ((dimensions of S) * d) x ((size at previous step)/d)
             # This ensures that the effective flip which happens at the middle of the spectrum due to thin SVD is adjusted for.  
            
            right_size = cld(size(psi,2),d);
            
            psi = reshape(psi_1,size(S,1)*d,right_size);

        end

    end

    return mps_state
end

state_to_mps (generic function with 1 method)

#### MPS to State

$\quad$ Given the MPS $\{A_1, A_2, A_3,\dots,A_N\}$, we can determine the state as follows

$\quad$ $\Psi_{\sigma_1,\sigma_2,\sigma_3,\dots,\sigma_N} = \sum_{\{a\}}A^{\sigma_1}_{1; a_1, a_2} A^{\sigma_2}_{2; a_2, a_3} \dots A^{\sigma_N}_{N; a_{N-1} a_{N}}$. Recall that $a_1$ and $a_N$ are trivial (dummy) indices of dimension $1$.

In [16]:
"""
    A function to perform the summation given above to obtain the state vector from the MPS.
    The first step is to convert it to the tensor ψ_{σ₁,σ₂,…}, which is then reshaped to a dᴺ dimensional Vector

"""

function mps_to_state(mps_state;return_tensor = false)

    mps_temp = mps_state; #using the left-bottom-right counting

    N = length(mps_temp);

    d = size(mps_temp[1])[2];

    state_tensor_size = ntuple(_ -> d, N);

    state_tensor  = fill(0.0,state_tensor_size...);

    G = CartesianIndices(state_tensor);

    @threads for g in G

        A = mps_temp[1][:,g[1],:];

        for l in 2:N
            
            f = g[l];

            B = mps_temp[l][:,f,:];

            @einsum M[i,j] := A[i,m]*B[m,j];

            A = M
        
        end
        
        state_tensor[g] = A[1]
    end

    if return_tensor

        return_obj = state_tensor;
    
    else
        
        return_obj = reshape(state_tensor,d^N);

    end
    return return_obj

end

mps_to_state (generic function with 1 method)

In [17]:
""" 

    A quick test to check : We generate a state, then generate the MPS from it and try to retrieve back the original state.
    Then do we get the correct state back?

"""
N = 8;
d = 2;
D = 10;

state_vec = rand(d^N); # The initial random state

mps_vec = state_to_mps(state_vec, N); # Convert that to MPS

state_vec_2 = mps_to_state(mps_vec); # Re-convert the MPS back to a state 

norm(state_vec .- state_vec_2) # Compute the error

7.894295549061714e-14

### Matrix Product Operators

$\quad$ Here we discuss the Matrix Product Operators that encode matrices. 

$\quad$ The general idea is the following

$\quad$ A matrix (say, Hamiltonian) can be expanded in terms of the basis states of local Hilbert spaces

$\quad$ $H = \sum_{\sigma, \tau} c^{\sigma, \tau}\ket{\tau}\bra{\sigma}$

$\quad$ $\;\;\;\; = \sum_{\sigma_1, \tau_1} \dots \sum_{\sigma_L , \tau_L} c^{\sigma_1,\sigma_2,\dots,\sigma_L}_{\tau_1, \tau_2,\dots, \tau_L} \ket{\tau_1, \tau_2,\dots, \tau_L}\bra{\sigma_1,\sigma_2,\dots,\sigma_L}$

$\quad$ The coefficients $c$ can be decomposed in terms of Matrix Products. For each site $i$ and every pair $\{ \ket{\tau_i}\bra{\sigma_i}\}$, we introduce a set of matrices $(W^{\sigma_i, \tau_i}_{i})_{w_{i-1},w_{i}}$.

$\quad$ These matrices then satisfy the property:

$\quad$ $\sum_{\mathbf{w}} (W^{\sigma_1 \tau_1}_1)_{w_0, w_1} (W^{\sigma_2 \tau_2}_2)_{w_1, w_2} \cdots (W^{\sigma_L \tau_L}_{L})_{w_{L-1}, w_L} = c^{\sigma_1,\sigma_2,\dots,\sigma_L}_{\tau_1, \tau_2,\dots, \tau_L}$

$\quad$ The general rule of thumb is that the Hamiltonians of the form $H = \sum_i X_i$ can be represented with $4-$ tensors of the form

$\quad$ $H_{i} = \begin{pmatrix} I & 0 \\ X & I \end{pmatrix}$

$\quad$ Similarly, the a nearest-neighbour Hamiltonian of the form $H = \sum_i X_i Y_{i + 1}$ should have the following $4-$ tensors

$\quad$ $H_{i} = \begin{pmatrix} I & 0 & 0 \\ X & 0 & 0 \\ 0 & Y & I \end{pmatrix}$

$\quad$ Now, on the other hand, if we have both nearest neighbour and local terms, i.e. $H = \sum_{i} X_{i}Y_{i + 1} + g Z_{i}$, then the local tensor is

$\quad$ $H_{i} = \begin{pmatrix} I & 0 & 0 \\ X & 0 & 0 \\ g Z & Y & I \end{pmatrix}$

$\quad$ The idea is similar for other Hamiltonians, but there are usually many more subtleties. For now, we work on a case by case basis.

$\quad$ The quantum Heisenberg chain, given by $H = J\sum_{i}\vec{\sigma}_{i}\vec{\sigma}_{i + 1} + h \sum_{i}\sigma^{z}_{i}$.

$\quad$ The constituent $4-$ tensor is given by:

$\quad$ $H_{i} = \begin{pmatrix} I & 0 & 0 & 0 & 0 \\ \sigma^{z} & 0 & 0 & 0 & 0 \\ \sigma^{+} & 0 & 0 & 0 & 0 \\ \sigma^{-} & 0 & 0 & 0 & 0  \\ h\sigma^{z} & J\sigma^{z} & \frac{J}{2}\sigma^{+} & \frac{J}{2}\sigma^{-} & I \end{pmatrix}$

In [18]:
function Heisenberg_MPO(J,h,N)

    σᶻ = [1 0 0; 0 0 0; 0 0 -1];

    σ⁺ = [0 sqrt(2) 0; 0 0 sqrt(2); 0 0 0];

    σ⁻ = [0 0 0; sqrt(2) 0 1; 0 sqrt(2) 1];

    I2 = I + zeros(3,3);

    Hi = fill(0.0,(5,3,5,3));

    Hi[1,:,1,:] = I2;

    Hi[2,:,1,:] = σᶻ;

    Hi[3,:,1,:] = σ⁺;

    Hi[4,:,1,:] = σ⁻;

    Hi[5,:,1,:] = -h * σᶻ;

    Hi[5,:,2,:] = J*σᶻ;

    Hi[5,:,3,:] = (J/2)*σ⁺;

    Hi[5,:,4,:] = (J/2)*σ⁻;

    Hi[5,:,5,:] = I2;

    Htensor = Any[];

    H = [Hi for i in 1:N];

    H[1] = Hi[size(Hi)[1]:size(Hi)[1],:,:,:];
    H[N] = Hi[:,:,1:1,:];
    
    return H 

end
    

Heisenberg_MPO (generic function with 1 method)

In [19]:
"""

    Function to convert an MPO to a Hamitlonian matrix.

"""

function MPO_to_Ham(mpo_operator)

    H_MPO = mpo_operator;

    N = length(H_MPO)

    T_res = H_MPO[1][1,:,:,:];

    for p in 2:N

        B = H_MPO[p];

        @einsum M1[i,j,k,l,m] := T_res[i,μ,j]*B[μ,k,l,m]

        M1 = permutedims(M1,(1,3,4,2,5));

        L = size(M1);

        M1 = reshape(M1,(L[1]*L[2],L[3],L[4]*L[5]));

        T_res = M1;

    end

    return T_res[:,1,:]

end

MPO_to_Ham (generic function with 1 method)

In [35]:
N = 4;
J = 1.;
h = 1.;

Heis_MPO = Heisenberg_MPO(J,h,N);

Heis_MAT = MPO_to_Ham(Heis_MPO);

#### Closing the Zipper : Expectation value

$\quad$ Here we discuss the method to ``close the zipper'' in order to compute the expectation values of the form $\bra{\phi} \hat{O} \ket{\psi}$. 

$\quad$ The closing the zipper mechanism follows practically as the contratction of the result of the previous contraction $C_{n-1}$ (rank $-3$), with

$\quad$ the MPS tensor at that index $B_n$ and the dual MPS tensor $A_n$ (like the zipper for states, both rank $-3$), with the additional $\textit{operator}$ MPO tensor

$\quad$ inserted at that position, which we denote by $O_n$ (rank $-4$).

$\quad$ The fist contraction step is to contract $B_n$ (which is below $O_n$), with $C_{n - 1}$. 

$\quad$ This consumes the left index of $B_n$ and the bottom index of $C_{n - 1}$. Both indices have the bond dimension $D$. 

$\quad$ The result is some tensor $M_1$ (repalcing $C_{n-1}$), which is a rank $-4$ tensor. 

$\quad$ $M_1$ has its' top leg connecting to the left leg of $A_n$, it's right leg connects to the left leg of $O_n$. 

$\quad$ The bottom index of $M_1$ connects to the bottom index of $O_n$, while it gets a free left index (which was the free right index of $B_n$).

$\quad$ The next step is to contract $M_1$ with $O_n$. This is a contraction of $2$ indices, i.e. the bottom and left indices of $O_n$ with the corresponding index of $M_1$.

$\quad$ $\textit{to be continued \dots}$.

In [20]:
"""
    Function to generate a random MPO 

"""

function rand_MPO(N,d,D; d1 = Normal(0,1))

    I2 = I  + zeros(d,d);

    H_MPO = Any[];

    HL = fill(0.0,(D,d,D,d));

    for l in 1:N 

        HL[1,:,1,:] = I2;

        for j in 2:(D-1)

            HL[j,:,1,:] = rand(d1,(d,d));
            HL[D,:,j,:] = rand(d1,(d,d));

        end 

        HL[D,:,D,:] = I2; 

        push!(H_MPO,HL)

    end 

    H_MPO[1] = H_MPO[1][size(HL,1):size(HL,1),:,:,:];

    H_MPO[N] = H_MPO[N][:,:,1:1,:];

    return H_MPO 

end

rand_MPO (generic function with 1 method)

In [21]:
"""

    MPO representation of an Identity matrix

"""

function Identity_MPO(N,d)

    I2 = I  + zeros(d,d);

    H_MPO = Any[];
    
    D = 2;

    HL = fill(0.0,(D,d,D,d));

    for l in 1:N 

        for j in 1:D

            HL[j,:,1,:] = I2;
            HL[D,:,j,:] = I2;

        end 

        HL[D,:,1,:] = I2/N;
        
        push!(H_MPO,HL)

    end 

    H_MPO[1] = H_MPO[1][size(HL,1):size(HL,1),:,:,:];

    H_MPO[N] = H_MPO[N][:,:,1:1,:];

    return H_MPO 

end

Identity_MPO (generic function with 1 method)

In [22]:
function ZIP_right(Cn_minus_1, An, Bn, On)
    
    # We count the legs from left to right. Thus, the first index of Cn_minus_1 must have bond dimension same as An, and the second index should
    # have bond dimension same as Bn

    Cn1 = Cn_minus_1;

    @einsum M1[i,j,l,m] := An[i,j,k]*Cn1[k,l,m]

    @einsum M2[i,l,m,n] := M1[i,j,k,l]*On[m,n,k,j]

    @einsum M3[i,j,k] := M2[i,m,j,n]*Bn[m,n,k]

    return M3
    
end

ZIP_right (generic function with 1 method)

In [23]:
function ZIP_left(Cn_minus_1, An, Bn, On)
    
    # We count the legs from left to right. Thus, the first index of Cn_minus_1 must have bond dimension same as An, and the second index should
    # have bond dimension same as Bn

    Cn1 = Cn_minus_1;

    @einsum M1[i,j,l,m] := Bn[i,j,k]*Cn1[k,l,m]

    @einsum M2[i,l,m,n] := M1[i,j,k,l]*On[k,j,m,n]

    @einsum M3[i,j,k] := M2[i,m,j,n]*An[m,n,k]

    return M3
    
end

ZIP_left (generic function with 1 method)

In [24]:
"""
    Here we define a function to compute the expectation value by the left zipper method, given the MPS and MPO.

"""

function exp_val_left(mps_state_1, mps_state_2, mpo_operator)

    N = length(mps_state_1);

    T_res = fill(1.0,(1,1,1));

    for l in 1:N 

        b_state = permutedims(conj.(mps_state_2[l]),(3,2,1));

        a_state = mps_state_1[l];

        op_tensor = mpo_operator[l];

        T_res = ZIP_left(T_res,a_state,b_state,op_tensor);

    end 

    return T_res[1,1,1]

end

exp_val_left (generic function with 1 method)

In [25]:
"""
    Here we test if this ``expectation value'' calculation gives us a proper number.

"""

N = 10;
d = 3;
D1 = 20;
D2 = 10;
D3 = 5;

V_state = mixed_canonical(rand_MPS(N,d,D1));

V_state_dag = mixed_canonical(rand_MPS(N,d,D2));

O_operator = rand_MPO(N,d,D3);

C = exp_val_left(V_state, V_state_dag, O_operator)

0.04113929392532821

$\quad$ Here we implement the action of an MPO on an MPS.

$\quad$ For this, we use the ``zip-up'' algorithm, which proceeds as follows.

$\quad$ 1.) The bottom index of the first MPO $O_1$ tensor is contracted with the top index of the first MPS tensor $A_1$. This gives us a rank $-3$ tensor $C_1$, with its' left leg $s_1$ being the  $\quad \quad \;$ top index of $O_1$ and the two right indices (which connect to the left indices of $O_2$ and $A_2$) are given by the two left indices of $O_1$ (denoted by $\mu_1) and $A_1$ (denoted  $\quad \quad \;$ by $\alpha_1$) respectively.

$\quad$ 2.) From $C_1$, we can perform an SVD. To do that we reshape $(\mu_1,\alpha_1)$ into one index and perform SVD. From this, we extract $U_1$ and $S_1 V_1^\dagger$. 

$\quad$ 3.) Then $U_1$ is treated as the resulting tensor of the MPS due to this contraction. And $S_1 V^\dagger_1$ is the next tensor to be contracted with $A_2$ and $O_2$

In [26]:
"""
    Function to perform one step of the zip-up algorithm

"""

function ZIP_up(Rn_minus_1, An, On)

    Rn = Rn_minus_1 ; # This is the resulting SV⁺ from the previous step. This is a 3-tensor.

    @einsum M1[i,j,k,l,m] := On[i,j,k,n]*An[l,n,m]

    M1 = permutedims(M1,(4,1,2,3,5));

    @einsum C[i,j,k,l] := Rn[i,a,b]*M1[b,a,j,k,l];

    return C
end

ZIP_up (generic function with 1 method)

In [27]:
D = 10;
d = 2;
Rn_minus_1 = rand(Normal(0,1),(D,D,D));
An = rand(Normal(0,1),(D,d,D));
On = rand(Normal(0,1),(D,d,D,d));

In [28]:
""" 
    Here we write the function that computes the action of an MPO on an MPS

""" 

function MPO_on_MPS(mpo_operator, mps_state)

    H_MPO = mpo_operator;

    A_MPS = mps_state;

    N = length(H_MPO);

    Rn = fill(1.0,(1,1,1));

    Res_MPS = Any[];

    for l in 1:N 

        An = A_MPS[l];

        On = H_MPO[l];

        C = ZIP_up(Rn, An, On);

        L = size(C);

        Cdim = reshape(C,(L[1]*L[2],L[3]*L[4]));

        U, S, V = svd(Cdim,full=false);

        push!(Res_MPS,reshape(U,(L[1],L[2],size(U)[2])));

        Sdiag = Diagonal(S)*V';

        Rn = reshape(Sdiag,(size(Sdiag)[1],L[3],L[4]));

    end

    return Res_MPS

end

MPO_on_MPS (generic function with 1 method)

In [29]:
N = 10;
D1 = 20;
D2 = 5;
d = 2;
R_MPS = rand_MPS(N,d,D1);
R_MPO = rand_MPO(N,d,D2);

In [30]:
C1 = MPO_on_MPS(R_MPO,R_MPS); # This gives the MPS obtained by the action of the MPO on the input MPS. 

## DMRG using MPO and MPS : $\textcolor{red}{Work\;\;In\;\;Progress}$
$\quad$ In this section, we discuss the one-site-update approach to DMRG using MPO and MPS.

$\quad$ The basic idea is to start with a random MPS $\ket{\psi}$, and find the expectation value $\bra{\psi}\hat{O}\ket{\psi}$. 

$\quad$ This expectation value is extremized with respect to the components of $\ket{\psi}$ to target some eigenvalue $\lambda$ via extremization of $\bra{\psi}\hat{O}\ket{\psi} - \lambda \braket{\psi \vert\psi}$

In [77]:
"""
    Here we write the 1-site update approach for DMRG to determine the ground state of a Hamiltonian

"""

function DMRG_1site(Ham_MPO,Max_Bond_Dim,Num_Sweeps)

    N = length(Ham_MPO);

    H = Ham_MPO;

    D = Max_Bond_Dim;

    NS = Num_Sweeps;

    # The first step is to generate tha random | ψ >

    d = size(H[1])[4]; # We assume that the physical dimensions are the same always. This can be generalized easily.

    ψ = mixed_canonical(rand_MPS(N,d,D)); # We ensure that the state | ψ > is cast into the mixed canonical (normalized) form.

    # For the 1-site update procedure, we fix a site l and contract from the left and right of l. The resulting matrix to be diagonalized to find the minimum energy.

    # The step now is to contract < ψ | H | ψ > from the left upto the index l, and then from the right. The results for each contraction are stored in the MPS CZ.

    # To begin, we cast CZ into a right canonical form

    CZ = [fill(1.0,(1,1,1)) for i in 1:N+1];

    for l in N:-1:1

        ψn = permutedims(conj.(ψ[l]),(3,2,1));

        CZ[l] = ZIP_right(CZ[l+1],ψ[l],ψn,H[l])

    end

    EV_list = Any[]; #This is the eigenvalue that we will store at each sweep.
    
    for J in 1:NS 

        ## First we do a right sweep.

        for l in 1:N 

            HM = H[l];

            CM = CZ[l];
            
            CM2 = CZ[l+1];

            @einsum M1[i,k,l,m,n] := CM[i,j,k]*HM[j,l,m,n]

            @einsum M2[i,j,k,m,n,p] := M1[i,j,k,l,m]*CM2[n,l,p];

            M2 = permutedims(M2,(1,3,6,2,4,5));

            L = size(M2);

            H = sparse(reshape(M2,(L[1]*L[2]*L[3],L[4]*L[5]*L[6])));

            #V = ψ[l];

            val, vecs = Arpack.eigs(H;nev=1,which=:SM); # Use ARPACK to obtain the eigenvalue and eigenvector

            push!(EV_list,val); # The eigenvalue stored from this iteration.

            M3 = reshape(vecs[:,1],(size(M2)[1]*size(M2)[2],size(M2)[3]));

            U, S, V = svd(M3,full=false);

            ψ[l] = reshape(U,(size(M2)[1],size(M2)[2],size(U)[2]));

            if l < N 
                G = ψ[l + 1];
                T = Diagonal(S)*V';
                @einsum V[i,j,k] = T[i,m]*G[m,j,k];
                ψ[l + 1] = V;
            end

            CZ[l + 1] = ZIP_left(CZ[l],ψ[l],permutedims(conj.(ψ[l]),(3,2,1)),H[l]);
        
        end

        ## Now we do a left sweep

        for l in N:-1:1 

            HM = H[l];

            CM = CZ[l];
            
            CM2 = CZ[l+1];

            @einsum M1[i,k,l,m,n] := CM[i,j,k]*HM[j,l,m,n]

            @einsum M2[i,j,k,m,n,p] := M1[i,j,k,l,m]*CM2[n,l,p];

            M2 = permutedims(M2,(1,3,6,2,4,5));

            L = size(M2);

            H = sparse(reshape(M2,(L[1]*L[2]*L[3],L[4]*L[5]*L[6])));

            #V = ψ[l];

            val, vecs = Arpack.eigs(H;nev=1,which=:SM); # Use ARPACK to obtain the eigenvalue and eigenvector

            push!(EV_list,val); # The eigenvalue stored from this iteration.

            M3 = reshape(vecs[:,1],(size(M2)[1]*size(M2)[2],size(M2)[3]));

            U, S, V = svd(M3,full=false);

            ψ[l] = reshape(U,(size(M2)[1],size(M2)[2],size(U)[2]));

            if l > 1 
                G = ψ[l - 1];
                T = Diagonal(S)*V';
                @einsum V[i,j,k] = T[i,m]*G[m,j,k];
                ψ[l - 1] = V;
            end

            CZ[l + 1] = ZIP_right(CZ[l+2],ψ[l],permutedims(conj.(ψ[l]),(3,2,1)),H[l]);
        
        end

    end

    return EV_list, ψ

end

DMRG_1site (generic function with 1 method)

In [68]:
"""
    Function that creates the MPO Hamiltonian for an XY chain with N spins and open boundary conditions.
"""
function XY_MPO(N)
    σp = zeros((2,2))
    σp[1,2] = 1
    σm = zeros((2,2))
    σm[2,1] = 1
    ## I_l
    I2 = I  + zeros(2,2)

    #Construct the MPO tensor
    Hl = zeros((4,2,4,2))
    Hl[1,:,1,:] = I2
    Hl[2,:,1,:] = σm
    Hl[3,:,1,:] = σp
    Hl[4,:,2,:] = -0.5*σp
    Hl[4,:,3,:] = -0.5*σm
    Hl[4,:,4,:] = I2
    ## H
    H = [Hl for l in 1:N]
    H[1] = Hl[size(Hl)[1]:size(Hl)[1],:,:,:]
    H[N] = Hl[:,:,1:1,:]
    
    return H

end

XY_MPO

In [81]:
N = 10;
D = 5;
d = 2;
CZ = [fill(1.0,(1,1,1)) for i in 1:N+1];
ψ = mixed_canonical(rand_MPS(N,d,D));
H = rand_MPO(N,d,D);

for l in N:-1:1

    ψn = permutedims(conj.(ψ[l]),(3,2,1));

    CZ[l] = ZIP_left(CZ[l+1],ψn,ψ[l],H[l])

end


AssertionError: AssertionError: size(M1, 3) == size(On, 1)

In [57]:
ψ = mixed_canonical(rand_MPS(N,d,D));

In [78]:
# parameters
N = 20
D = 10 #bond dimension for DMRG
Nsweeps = 3 #nr of sweeps

# Hamiltonian MPO
H = XY_MPO(N)

# DMRG
E_list,M = DMRG_1site(H,D,Nsweeps)

AssertionError: AssertionError: size(M1, 2) == size(On, 2)